Ideas for Future:

Rather than scrape ump and venue data at run time, run a function each day that runs once, scrapes websites and saves data to a csv for the program to pull from.

Import Necessary Libraries

In [9]:
import pandas as pd
import tempfile
import os
from datetime import date as d
from datetime import timedelta

Set Up API

In [10]:
import statsapi as api

Set Up Beautiful Soup

In [11]:
import re
import requests
from bs4 import BeautifulSoup, SoupStrainer

Set Up Selenium

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

For killing background chrome processes:

In [13]:
#import psutil
#import subprocess
#import signal

#def kill_chrome_processes():
#    for proc in psutil.process_iter():
#        if 'chrome' in proc.name() or 'chromedriver' in proc.name:
#            proc.kill()

Input Player Name and Date


NEED TO CHECK AND FIX ALL CELLS: IF STATS FOR SOMETHING DO NOT EXIST NEED TO DEAL WITH THAT!

ALSO NEED TO ENSURE CORRECT VALUES ARE PASSED IN FOR COMPUTING CORRECT VALUES

NEED TO CHECK PLAYER AVG OVER X GAMES - NEED TO COMPUTE PLAYER AVG OVER LAST GAME FOR PAST SEASONS AS WELL

NEED TO ENSURE THAT ALL DATA IS OF CORRECT TYPE, CORRECT VALUES, CORRECT YEARS, AND IF NONE NEEDS TO BE HANDLED

FIX CURRENT SET UP FOR COMPUTING AVERAGES. SHIT IS BUGGY AS FUCK

IF IT APPEARS TO BE GOOD: THERE WILL BE :P NEXT TO IT. IF NEEDS ADJUSTMENTS, VERIFICATION, OR REWORK, :( NEXT TO IT

ESPECIALLY DECIDE HOW TO DEAL WITH EDGE CASES AND DIVISION BY 0.

WALKS, HIT BY PITCH, AND SAC FLIES DO NOT COUNT AS AN AT BAT

TOTAL BASES ARE ONLY BASES REACHED VIA HITS.

In [14]:
first_name = input("Enter Player First Name: ")
last_name = input('Enter Player Last Name: ')
#date = input('Enter Date: ')
date = '2024-07-31'
og_date = date
year = int(date[:4])
last_year = year - 1
print(year)
player_name = f'{first_name.lower()} {last_name.lower()}'


2024


fetch player_id :P

In [15]:
player_info = api.lookup_player(player_name, season=year)
player_name = player_info[0]['fullName']
player_id = player_info[0]['id']
team_id = player_info[0]['currentTeam']['id']
team_info = api.lookup_team(team_id, season=year)
team_name = team_info[0]['name']
team_abv = team_info[0]['fileCode'].upper()

if len(team_abv) < 3:
    team_abv = f'{team_abv}{team_info[0]['teamName'][0]}'

print(player_name)
print(player_id)
print(team_name)
print(team_abv)
print(team_id)

Shohei Ohtani
660271
Los Angeles Dodgers
LAD
119


Gather batter_hand, batter_age, , batter_strike_zone_top, batter_strike_zone_bottom :P

In [16]:
person_info = api.get('people',{'personIds':player_id, 'season':year,'sportId':1})
batter_hand = person_info['people'][0]['batSide']['code']
batter_age = person_info['people'][0]['currentAge']
batter_strike_zone_top = person_info['people'][0]['strikeZoneTop']
batter_strike_zone_bottom = person_info['people'][0]['strikeZoneBottom']
batter_starting_year = int(person_info['people'][0]['mlbDebutDate'][:4])
batter_year = year - batter_starting_year


print(batter_age)
print(batter_year)
print(batter_hand)
print(batter_strike_zone_top)
print(batter_strike_zone_bottom)

30
6
L
3.51
1.61


fetch game_pk, home, opp_id, venue_id, sPitcher_id, sPitcher_hand :P

In [17]:
game_info = api.schedule(date=og_date,team=team_id,sportId=1)
game_pk = game_info[0]['game_id']
home_team_id = game_info[0]['home_id']
if home_team_id == team_id:
    home = True
    opp_id = game_info[0]['away_id']
    sPitcher_name = game_info[0]['away_probable_pitcher']
else:
    home = False
    opp_id = game_info[0]['home_id']
    sPitcher_name = game_info[0]['home_probable_pitcher']

if sPitcher_name == '':
    sPitcher_name = None
    sPitcher_id = None
    sPitcher_hand = None
else:
    sPitcher_info = api.lookup_player(sPitcher_name)
    sPitcher_id = sPitcher_info[0]['id']
    sPitcher_info_2 = api.get('people',{'personIds':sPitcher_id, 'season':year,'sportId':1})
    sPitcher_hand = sPitcher_info_2['people'][0]['pitchHand']['code']

venue_id = game_info[0]['venue_id']
venue_name = game_info[0]['venue_name']

if game_info[0]['status'] == 'Final':
    game_status = True
else:
    game_status = False

opp_info = api.lookup_team(opp_id, season=year)

opp_name = opp_info[0]['name']
opp_abv = opp_info[0]['fileCode'].upper()

if len(opp_abv) < 3:
    opp_abv = f'{opp_abv}{opp_info[0]['teamName'][0]}'

print(game_status)
print(game_pk)
print(opp_name)
print(opp_abv)
print(opp_id)
print(sPitcher_name)
print(sPitcher_id)
print(sPitcher_hand)
print(venue_name)
print(venue_id)
print(home)

True
745386
San Diego Padres
SDP
135
Dylan Cease
656302
R
Petco Park
2680
False


fetch player game stats: temp, wind_speed, hp_ump_id, game_time. Fetch list of proj_ump_ids if Home Plate Umpire not yet specified :(

What should happen when this data isnt uploaded yet?

In [18]:
game_boxscore_info = api.boxscore_data(gamePk=game_pk)

temp = None
wind_speed = None

for x in game_boxscore_info['gameBoxInfo']:
    if x.get('label') == 'Weather':
        temp_info = x['value']
        temps = re.findall(r'\d+', temp_info)
        if temps:
            temp = int(temps[0])
            

    if x.get('label') == 'Wind':
        wind_speed_info = x['value']
        winds = re.findall(r'\d+', wind_speed_info)
        if winds:
            wind_speed = int(winds[0])
            

    #if x.get('label') == 'First pitch':
    #    time_str = x['value'].strip('.')
    #    time_part, period = time_str.split(' ')
    #    hour_str, min_str = time_part.split(':')
    
    #    if period.lower() == 'am':
    #        game_time_str = f'{hour_str}{min_str}'
        
    #    elif period.lower() == 'pm':
    #        game_time_str = f'{int(hour_str) + 12}{min_str}'
        
    #    game_time = int(game_time_str)


    if x.get('label') == 'Umpires':
        ump_info = x['value']
        proj_ump_ids = []
        proj_umps = []
        if 'HP:' in ump_info:
            pattern = r'HP: (.*?)\.'
            match = re.search(pattern, ump_info)
            if match:
                hp_ump = match.group(1)
                hp_ump_info = api.get('jobs_umpires', {})
                for person in hp_ump_info['roster']:
                    if person['person'].get('fullName') == hp_ump:
                        hp_ump_id = person['person'].get('id')
                        proj_umps.append(hp_ump)
                        proj_ump_ids.append(hp_ump_id)

        elif ump_info:
            umps = re.findall(r'[\w\s]+', ump_info)
            umps = [ump.strip() for ump in umps if ump.strip]
            ump_list = list(umps) + [None]*(4-len(umps))
            umps_tuple = tuple(ump_list[:4])
            for ump in umps_tuple:
                ump_info = api.get('jobs_umpires',{})
                for person in ump_info['roster']:
                    if person['person'].get('fullName') == ump:
                        ump_id = person['person'].get('id')
                        proj_umps.append(ump)
                        proj_ump_ids.append(ump_id)

game_datetime = api.schedule(game_id=game_pk)[0]['game_datetime'][11:16]
hour_str, min_str = game_datetime.split(':')
game_hour = int(hour_str) - 4
if game_hour < 0:
    game_hour += 24

game_time = int(f'{str(game_hour)}{min_str}')

print(temp)
print(wind_speed)
print(game_time)
print(proj_ump_ids)
print(proj_umps)



73
8
2040
[521251]
['Ben May']


fetch player game data: :P
- batting position

In [19]:
game_content = api.boxscore_data(gamePk=game_pk)
if home == True:
    batters_info = game_content['homeBatters']
else:
    batters_info = game_content['awayBatters']

batting_position = None

for batter in batters_info:
    if batter.get('personId') == player_id:
        batting_position = int(batter.get('battingOrder')[0])

print (batting_position)

1


fetch game player stats :P
- If game_status == True
    - gather player stats
- If game_status == False
    - set player stats to None

In [20]:
if game_status == True:
    game_stats_info = api.get('person_stats',{'personId':player_id,'gamePk':game_pk})
    game_stats = game_stats_info['stats'][0]['splits'][2]['stat']
    pa = game_stats.get('plateAppearances')
    ab = game_stats.get('atBats')
    h = game_stats.get('hits')
    r = game_stats.get('runs')
    rbi = game_stats.get('rbi')
    hr = game_stats.get('homeRuns')
    tb = game_stats.get('totalBases')
    k = game_stats.get('strikeOuts')
    bb = game_stats.get('baseOnBalls')

else:
    pa = None
    ab = None
    h = None
    r = None
    rbi = None
    hr = None
    tb = None
    k = None
    bb = None

batter_game_stats_dict = {'pa': pa, 'ab': ab, 'h': h, 'r': r, 'rbi': rbi, 'hr': hr, 'tb': tb, 'k': k, 'bb': bb}

batter_game_stats_df = pd.DataFrame([batter_game_stats_dict])

print(batter_game_stats_df)


   pa  ab  h  r  rbi  hr  tb  k  bb
0   4   3  0  0    0   0   0  0   1


Determine Whether Most Recent Streak Averages, or Historical Streak Averages are Needed for Player

Format Player URL for baseballreference.com

In [21]:
initial = last_name[:1].lower()

full_name = f'{first_name} {last_name}'.lower()

url = f'https://www.baseball-reference.com/players/{initial}/'
headers = {'User_Agent': 'Mozilla/5.0'}

response = requests.get(url, headers=headers)
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')

initial_section = soup.find('div', class_="section_content", id="div_players_")

if initial_section:
    active_players = initial_section.find_all('b')
    for players in active_players:
        player = players.find('a')
        years_playing = players.get_text()[(len(players.get_text()) - 11):]
        if full_name == player.get_text().lower():
            start_year = int(years_playing[1:5])
            end_year = year
            if year in range(start_year, end_year + 1):
                player_link = player['href']
                formatted_player_key = player_link[11:20]

print(formatted_player_key)

ohtansh01


Collect Data for Games Played Up to Desired Date
    - For entirely current year, or entirely last year

Format Player Game Data URL

In [22]:
season_start_date = api.get('season', {'seasonId': year, 'sportId': 1})['seasons'][0]['regularSeasonStartDate']

if date == season_start_date:
    date = api.get('season', {'seasonId': last_year, 'sportId': 1})['seasons'][0]['postSeasonEndDate']
    search_url = f'https://www.baseball-reference.com/players/gl.fcgi?id={formatted_player_key}&t=b&year={last_year}#batting_gamelogs'
else:
    search_url = f'https://www.baseball-reference.com/players/gl.fcgi?id={formatted_player_key}&t=b&year={year}#batting_gamelogs'

Set Correct Date

In [23]:
day = int(date[8:10])
month = int(date[5:7])

date_dt = d(year, month, day)

new_date_dt = date_dt - timedelta(days=1)

date = new_date_dt.strftime("%Y-%m-%d")

day = int(date[8:10])
month = int(date[5:7])

print(date)
print(day)
print(month)

2024-07-30
30
7


Scrape Player Game Data

In [24]:
response = requests.get(search_url)
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')

table_rows = soup.find_all('tr', id=lambda x: x and x.startswith('batting_gamelogs'))

if table_rows:

    data = []

    for row in table_rows:

        row_data = {}

        th = row.find('th')
        
        row_num = th.text
        
        row_data['game_num'] = row_num

        for td in row.find_all('td'):

            stat_type = td['data-stat']

            if stat_type == 'date_game':
                stat_value = td['csk'][:10]
            else:
                stat_value = td.text

            row_data[stat_type] = stat_value

        if row_data:

            game_day = int(row_data['date_game'][8:10])
            game_month = int(row_data['date_game'][5:7])

            if game_month < month:
                    data.append(row_data)
            elif game_month == month:
                if game_day <= day:
                     data.append(row_data)

    if data:
        df = pd.DataFrame(data)

        df.rename(columns={
            'batting_avg': 'BA',
            'onbase_perc': 'OBP',
            'slugging_perc': 'SLG',
        }, inplace=True)

        all_columns = df.columns.tolist()

        columns_to_keep = ['game_num', 'date_game', 'opp_ID', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'HBP']

        columns_to_drop = [column for column in all_columns if column not in columns_to_keep]

        correct_columns_df = df.drop(columns_to_drop, axis=1)

        columns_to_convert_int = ['game_num', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'HBP']

        columns_to_convert_float = ['BA', 'OBP', 'SLG']

        for column in columns_to_convert_int:
            correct_columns_df[column] = correct_columns_df[column].astype(int)

        for column in columns_to_convert_float:
            correct_columns_df[column] = correct_columns_df[column].astype(float)

        correct_columns_df['TB'] = correct_columns_df.apply(lambda row: (row['H'] - row['2B'] - row['3B'] - row['HR']) + 2*row['2B'] + 3*row['3B'] + 4*row['HR'], axis=1)

        column_order = ['game_num', 'date_game', 'opp_ID', 'PA', 'AB', 'H', 'R', 'RBI', '2B', '3B', 'HR', 'TB', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'HBP']
        correct_columns_df = correct_columns_df[column_order]
    
    else:
        correct_columns_df = None

else:
    correct_columns_df = None

print(correct_columns_df)


     game_num   date_game opp_ID  PA  AB  H  R  RBI  2B  3B  HR  TB  BB  SO  \
0           1  2024-03-20    SDP   5   5  2  0    1   0   0   0   2   0   0   
1           2  2024-03-21    SDP   6   5  1  1    1   0   0   0   1   0   0   
2           3  2024-03-28    STL   4   3  2  1    0   1   0   0   3   1   1   
3           4  2024-03-29    STL   4   4  0  0    0   0   0   0   0   0   1   
4           5  2024-03-30    STL   6   5  1  1    0   0   0   0   1   1   2   
..        ...         ...    ...  ..  .. .. ..  ...  ..  ..  ..  ..  ..  ..   
100       101  2024-07-25    SFG   5   4  2  1    1   1   0   1   6   1   0   
101       102  2024-07-26    HOU   4   3  2  0    0   1   0   0   3   1   0   
102       103  2024-07-27    HOU   5   3  2  2    2   0   0   1   5   2   0   
103       104  2024-07-28    HOU   5   4  0  1    0   0   0   0   0   1   2   
104       105  2024-07-30    SDP   5   4  0  1    0   0   0   0   0   1   2   

        BA    OBP    SLG  HBP  
0    0.400  0.400  

Season Averages Up to Desired Date

In [25]:
season_stats = {'games_played': None, 'pa_per_game': None, 'ab_per_game': None, 'h_per_game': None, 'r_per_game': None, 'rbi_per_game': None, 'hr_per_game': None, 
                'tb_per_game': None, 'bb_per_game': None, 'k_per_game': None, 'ba_szn': None, 'obp_szn': None, 'slg_szn': None}

if correct_columns_df is not None:

    game_data = correct_columns_df.iloc[-1]

    games_played = game_data['game_num']

    print(games_played)

    season_stats['games_played'] = games_played
    season_stats['pa_per_game'] = correct_columns_df['PA'].mean()
    season_stats['ab_per_game'] = correct_columns_df['AB'].mean()
    season_stats['h_per_game'] = correct_columns_df['H'].mean()
    season_stats['r_per_game'] = correct_columns_df['R'].mean()
    season_stats['rbi_per_game'] = correct_columns_df['RBI'].mean()
    season_stats['hr_per_game'] = correct_columns_df['HR'].mean()
    season_stats['tb_per_game'] = correct_columns_df['TB'].mean()
    season_stats['bb_per_game'] = correct_columns_df['BB'].mean()
    season_stats['k_per_game'] = correct_columns_df['SO'].mean()
    season_stats['ba_szn'] = game_data['BA']
    season_stats['obp_szn'] = game_data['OBP']
    season_stats['slg_szn'] = game_data['SLG']

season_stats_df = pd.DataFrame([season_stats])

print(season_stats_df)

105
   games_played  pa_per_game  ab_per_game  h_per_game  r_per_game  \
0           105     4.571429     3.914286    1.219048    0.790476   

   rbi_per_game  hr_per_game  tb_per_game  bb_per_game  k_per_game  ba_szn  \
0       0.72381     0.304762      2.47619     0.580952    1.038095   0.311   

   obp_szn  slg_szn  
0    0.402    0.633  


Streak Average Data Collection

In [26]:
if games_played < 30:
    last_season_end_date = api.get('season', {'seasonId': last_year, 'sportId': 1})['seasons'][0]['postSeasonEndDate']

    season_end_day = int(last_season_end_date[8:10])
    season_end_month = int(last_season_end_date[5:7])

    search_url = f'https://www.baseball-reference.com/players/gl.fcgi?id={formatted_player_key}&t=b&year={last_year}#batting_gamelogs'

    response = requests.get(search_url)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, 'html.parser')

    table_rows = soup.find_all('tr', id=lambda x: x and x.startswith('batting_gamelogs'))

    if table_rows:

        data = []

        for row in table_rows:

            row_data = {}

            th = row.find('th')
            
            row_num = th.text
            
            row_data['game_num'] = row_num

            for td in row.find_all('td'):

                stat_type = td['data-stat']

                if stat_type == 'date_game':
                    stat_value = td['csk'][:10]
                else:
                    stat_value = td.text

                row_data[stat_type] = stat_value

            if row_data:

                game_day = int(row_data['date_game'][8:10])
                game_month = int(row_data['date_game'][5:7])

                if game_month < season_end_month:
                        data.append(row_data)
                elif game_month == season_end_month:
                    if game_day <= season_end_day:
                        data.append(row_data)

        df = pd.DataFrame(data)

        df.rename(columns={
            'batting_avg': 'BA',
            'onbase_perc': 'OBP',
            'slugging_perc': 'SLG',
        }, inplace=True)

        all_columns = df.columns.tolist()

        columns_to_keep = ['game_num', 'date_game', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'HBP']

        columns_to_drop = [column for column in all_columns if column not in columns_to_keep]

        last_season_df = df.drop(columns_to_drop, axis=1)

        columns_to_convert_int = ['game_num', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'HBP']

        columns_to_convert_float = ['BA', 'OBP', 'SLG']

        for column in columns_to_convert_int:
            last_season_df[column] = last_season_df[column].astype(int)

        for column in columns_to_convert_float:
            last_season_df[column] = last_season_df[column].astype(float)

        last_season_df['TB'] = last_season_df.apply(lambda row: (row['H'] - row['2B'] - row['3B'] - row['HR']) + 2*row['2B'] + 3*row['3B'] + 4*row['HR'], axis=1)

        column_order = ['game_num', 'date_game', 'PA', 'AB', 'H', 'R', 'RBI', '2B', '3B', 'HR', 'TB', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'HBP']
        last_season_df = last_season_df[column_order]

    print(last_season_df)




Last 7 Averages:

In [27]:
if games_played < 7:
    carry_over_games = 7 - games_played
    carry_over_game_num = len(last_season_df) - carry_over_games
    last_season_rows = last_season_df.iloc[carry_over_game_num:]
    current_season_rows = correct_columns_df.iloc[:games_played]
    last_7_df = pd.concat([last_season_rows, current_season_rows], ignore_index=True)
else:
    last_7_game_num = games_played - 7
    last_7_df = correct_columns_df.iloc[last_7_game_num:]

print(last_7_df)

last_7_stats = {}

last_7_stats['pa_last_7'] = last_7_df['PA'].mean()
last_7_stats['ab_last_7'] = last_7_df['AB'].mean()
last_7_stats['h_last_7'] = last_7_df['H'].mean()
last_7_stats['r_last_7'] = last_7_df['R'].mean()
last_7_stats['rbi_last_7'] = last_7_df['RBI'].mean()
last_7_stats['hr_last_7'] = last_7_df['HR'].mean()
last_7_stats['tb_last_7'] = last_7_df['TB'].mean()
last_7_stats['bb_last_7'] = last_7_df['BB'].mean()
last_7_stats['k_last_7'] = last_7_df['SO'].mean()
last_7_stats['ba_last_7'] = last_7_df['H'].sum() / last_7_df['AB'].sum()
last_7_stats['obp_last_7'] = (last_7_df['H'].sum() + last_7_df['BB'].sum() + last_7_df['HBP'].sum()) / last_7_df['PA'].sum()
last_7_stats['slg_last_7'] = last_7_df['TB'].sum() / last_7_df['AB'].sum()

last_7_avg_df = pd.DataFrame([last_7_stats])

print(last_7_avg_df)

     game_num   date_game opp_ID  PA  AB  H  R  RBI  2B  3B  HR  TB  BB  SO  \
98         99  2024-07-23    SFG   5   5  2  0    3   1   0   0   3   0   3   
99        100  2024-07-24    SFG   4   4  0  0    0   0   0   0   0   0   3   
100       101  2024-07-25    SFG   5   4  2  1    1   1   0   1   6   1   0   
101       102  2024-07-26    HOU   4   3  2  0    0   1   0   0   3   1   0   
102       103  2024-07-27    HOU   5   3  2  2    2   0   0   1   5   2   0   
103       104  2024-07-28    HOU   5   4  0  1    0   0   0   0   0   1   2   
104       105  2024-07-30    SDP   5   4  0  1    0   0   0   0   0   1   2   

        BA    OBP    SLG  HBP  
98   0.314  0.400  0.632    0  
99   0.310  0.396  0.626    0  
100  0.312  0.399  0.635    0  
101  0.315  0.402  0.638    0  
102  0.318  0.406  0.645    0  
103  0.314  0.404  0.639    0  
104  0.311  0.402  0.633    0  
   pa_last_7  ab_last_7  h_last_7  r_last_7  rbi_last_7  hr_last_7  tb_last_7  \
0   4.714286   3.857143  1.142

Last 15 Averages

In [28]:
if games_played < 15:
    carry_over_games = 15 - games_played
    carry_over_game_num = len(last_season_df) - carry_over_games
    last_season_rows = last_season_df.iloc[carry_over_game_num:]
    current_season_rows = correct_columns_df.iloc[:games_played]
    last_15_df = pd.concat([last_season_rows, current_season_rows], ignore_index=True)
else:
    last_15_game_num = games_played - 15
    last_15_df = correct_columns_df.iloc[last_15_game_num:]

print(last_15_df)

last_15_stats = {}

last_15_stats['pa_last_15'] = last_15_df['PA'].mean()
last_15_stats['ab_last_15'] = last_15_df['AB'].mean()
last_15_stats['h_last_15'] = last_15_df['H'].mean()
last_15_stats['r_last_15'] = last_15_df['R'].mean()
last_15_stats['rbi_last_15'] = last_15_df['RBI'].mean()
last_15_stats['hr_last_15'] = last_15_df['HR'].mean()
last_15_stats['tb_last_15'] = last_15_df['TB'].mean()
last_15_stats['bb_last_15'] = last_15_df['BB'].mean()
last_15_stats['k_last_15'] = last_15_df['SO'].mean()
last_15_stats['ba_last_15'] = last_15_df['H'].sum() / last_15_df['AB'].sum()
last_15_stats['obp_last_15'] = (last_15_df['H'].sum() + last_15_df['BB'].sum() + last_15_df['HBP'].sum()) / last_15_df['PA'].sum()
last_15_stats['slg_last_15'] = last_15_df['TB'].sum() / last_15_df['AB'].sum()

last_15_avg_df = pd.DataFrame([last_15_stats])

print(last_15_avg_df)

     game_num   date_game opp_ID  PA  AB  H  R  RBI  2B  3B  HR  TB  BB  SO  \
90         91  2024-07-11    PHI   4   4  0  0    0   0   0   0   0   0   2   
91         92  2024-07-12    DET   5   5  1  0    1   1   0   0   2   0   1   
92         93  2024-07-13    DET   6   4  2  3    2   0   1   1   7   2   1   
93         94  2024-07-14    DET   4   4  2  0    0   0   0   0   2   0   0   
94         95  2024-07-19    BOS   4   4  1  1    0   1   0   0   2   0   3   
95         96  2024-07-20    BOS   6   4  1  1    0   1   0   0   2   2   0   
96         97  2024-07-21    BOS   4   3  1  1    1   0   0   1   4   1   1   
97         98  2024-07-22    SFG   4   3  0  0    0   0   0   0   0   1   2   
98         99  2024-07-23    SFG   5   5  2  0    3   1   0   0   3   0   3   
99        100  2024-07-24    SFG   4   4  0  0    0   0   0   0   0   0   3   
100       101  2024-07-25    SFG   5   4  2  1    1   1   0   1   6   1   0   
101       102  2024-07-26    HOU   4   3  2  0    0 

Last 30 Averages

In [29]:
if games_played < 30:
    carry_over_games = 30 - games_played
    carry_over_game_num = len(last_season_df) - carry_over_games
    last_season_rows = last_season_df.iloc[carry_over_game_num:]
    current_season_rows = correct_columns_df.iloc[:games_played]
    last_30_df = pd.concat([last_season_rows, current_season_rows], ignore_index=True)
else:
    last_30_game_num = games_played - 30
    last_30_df = correct_columns_df.iloc[last_30_game_num:]

print(last_30_df)

last_30_stats = {}

last_30_stats['pa_last_30'] = last_30_df['PA'].mean()
last_30_stats['ab_last_30'] = last_30_df['AB'].mean()
last_30_stats['h_last_30'] = last_30_df['H'].mean()
last_30_stats['r_last_30'] = last_30_df['R'].mean()
last_30_stats['rbi_last_30'] = last_30_df['RBI'].mean()
last_30_stats['hr_last_30'] = last_30_df['HR'].mean()
last_30_stats['tb_last_30'] = last_30_df['TB'].mean()
last_30_stats['bb_last_30'] = last_30_df['BB'].mean()
last_30_stats['k_last_30'] = last_30_df['SO'].mean()
last_30_stats['ba_last_30'] = last_30_df['H'].sum() / last_30_df['AB'].sum()
last_30_stats['obp_last_30'] = (last_30_df['H'].sum() + last_30_df['BB'].sum() + last_30_df['HBP'].sum()) / last_30_df['PA'].sum()
last_30_stats['slg_last_30'] = last_30_df['TB'].sum() / last_30_df['AB'].sum()

last_30_avg_df = pd.DataFrame([last_30_stats])

print(last_30_avg_df)

     game_num   date_game opp_ID  PA  AB  H  R  RBI  2B  3B  HR  TB  BB  SO  \
75         76  2024-06-22    LAA   5   4  1  1    2   0   0   1   4   1   0   
76         77  2024-06-24    CHW   5   3  0  0    1   0   0   0   0   1   2   
77         78  2024-06-25    CHW   5   4  2  2    2   0   0   1   5   1   2   
78         79  2024-06-26    CHW   4   2  1  2    1   0   0   1   4   2   0   
79         80  2024-06-28    SFG   4   3  1  0    0   0   0   0   1   1   1   
80         81  2024-06-29    SFG   6   4  1  2    1   0   0   1   4   2   2   
81         82  2024-06-30    SFG   5   5  0  0    0   0   0   0   0   0   3   
82         83  2024-07-02    ARI   5   5  3  2    2   1   0   1   7   0   2   
83         84  2024-07-03    ARI   4   4  1  1    0   0   0   0   1   0   1   
84         85  2024-07-04    ARI   4   3  0  0    0   0   0   0   0   1   3   
85         86  2024-07-05    MIL   5   5  0  0    0   0   0   0   0   0   3   
86         87  2024-07-06    MIL   5   2  2  2    1 

Merge Streak Data

In [30]:
streak_data_df = pd.concat([last_7_avg_df, last_15_avg_df, last_30_avg_df], axis=1)
print(streak_data_df)

   pa_last_7  ab_last_7  h_last_7  r_last_7  rbi_last_7  hr_last_7  tb_last_7  \
0   4.714286   3.857143  1.142857  0.714286    0.857143   0.285714   2.428571   

   bb_last_7  k_last_7  ba_last_7  ...  h_last_30  r_last_30  rbi_last_30  \
0   0.857143  1.428571   0.296296  ...   1.066667   0.766667          0.7   

   hr_last_30  tb_last_30  bb_last_30  k_last_30  ba_last_30  obp_last_30  \
0    0.333333    2.433333         0.8   1.433333    0.283186     0.410072   

   slg_last_30  
0     0.646018  

[1 rows x 36 columns]


Merge Season Averages and Streak Data



In [31]:
batter_season_stats_df = pd.concat([streak_data_df, season_stats_df], axis=1)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

print(batter_season_stats_df)

   pa_last_7  ab_last_7  h_last_7  r_last_7  rbi_last_7  hr_last_7  tb_last_7  \
0   4.714286   3.857143  1.142857  0.714286    0.857143   0.285714   2.428571   

   bb_last_7  k_last_7  ba_last_7  obp_last_7  slg_last_7  pa_last_15  \
0   0.857143  1.428571   0.296296    0.424242     0.62963    4.666667   

   ab_last_15  h_last_15  r_last_15  rbi_last_15  hr_last_15  tb_last_15  \
0    3.866667   1.066667   0.733333     0.666667    0.266667         2.4   

   bb_last_15  k_last_15  ba_last_15  obp_last_15  slg_last_15  pa_last_30  \
0         0.8   1.333333    0.275862          0.4      0.62069    4.633333   

   ab_last_30  h_last_30  r_last_30  rbi_last_30  hr_last_30  tb_last_30  \
0    3.766667   1.066667   0.766667          0.7    0.333333    2.433333   

   bb_last_30  k_last_30  ba_last_30  obp_last_30  slg_last_30  games_played  \
0         0.8   1.433333    0.283186     0.410072     0.646018           105   

   pa_per_game  ab_per_game  h_per_game  r_per_game  rbi_per_game 

Fetch Starting Pitcher Season Data


MASSIVE:

PLAN FOR FUTURE FOR PITCHER, BVP, BVOPP, OPP, BVB Data

DESIGN THE 'Starting Pitcher Efficiency Index'

Step 1: SP Data
    - Gather Data such as:
        - ip per game
        - batters faced per game
        - plate appearances per game
        - pitches per game
        - at bats per game

Step 2: Include these new stat categories:
    - Batter Vs Opponent - Previous Batter stats from each game against the opponent this season
    - Team Vs Opponent Stats - Avg Team Stats for Each Final Boxscore of Previous Season Matchups
    - Pitcher vs Team Stats - Avg Pitcher Stats Vs Batter's team from previous mathcups
    - Batter Vs Pitcher - Batter Vs Pitcher Stats from IP or PA from each game featuring the two
    - Batter Vs Bullpen (Batter Vs Team - Batter Vs Pitcher)

    Goal is to determine how valuable the batter's plate appearences with the starting pitcher compared to the batter's plate appearances with the relievers and closers.
    What percentage of hits come from the starting pitcher vs the relievers. Want to try to predict the number of plate appearnaces the batter will get during a game with this starting pitcher, and then use this value to predict hits by comparing the likelihood the batter will get a hit within the plate appearances against the starting pitcher and the likelihood the batter will get a hit within the plate appearances against the relievers and closers. Then using these averages, predict how likely the batter is to get a hit in this game. Basically instead of just using starting pitcher, want to treat both the starting pitcher and other pitchers as an entire game to predict hits. This will be done by leveraging the innings pitched by the starter and other pitchers with the stats of starter v batter and bullpen v batter

Stats to Collect:
- sp_season_stats -- starting pitcher season averages up to the desired date. x
- bv_sp_stats -- direct batter vs starting pitcher stats measured by per plate appearance. x
- sp_v_team -- starting pitcher game stats in games vs team this season.
- bv_opp_stats -- batter game stats in games vs opp this season. x
- opp_stats -- opposing team per game stats 
- bv_opp_sp_stats -- batter game stats in games vs opp where the starting pitcher started this season. x
- team_v_opp_stats -- overall team stats in games vs opp this season.

Format Pitcher URL

In [32]:
sp_name_parts = sPitcher_name.split()

sp_first_name = sp_name_parts[0].lower()
sp_last_name = sp_name_parts[1].lower()

initial = sp_last_name[:1]

full_name = f'{sp_first_name} {sp_last_name}'

url = f'https://www.baseball-reference.com/players/{initial}/'
headers = {'User_Agent': 'Mozilla/5.0'}

response = requests.get(url, headers=headers)
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')

initial_section = soup.find('div', class_="section_content", id="div_players_")

if initial_section:
    active_players = initial_section.find_all('b')
    for players in active_players:
        player = players.find('a')
        years_playing = players.get_text()[(len(players.get_text()) - 11):]
        if full_name == player.get_text().lower():
            start_year = int(years_playing[1:5])
            end_year = year
            if year in range(start_year, end_year + 1):
                player_link = player['href']
                #formatted_pitcher_key = player_link[11:20]

pattern = r'/players/[a-z]/([a-z]{1,5}[a-z]{2}\d{2})\.shtml'

match = re.search(pattern, player_link)

if match:
    formatted_pitcher_key = match.group(1)

print(formatted_pitcher_key)

ceasedy01


In [33]:
season_start_date = api.get('season', {'seasonId': year, 'sportId': 1})['seasons'][0]['regularSeasonStartDate']

search_url = f'https://www.baseball-reference.com/players/gl.fcgi?id={formatted_pitcher_key}&t=p&year={year}#pitching_gamelogs'

response = requests.get(search_url)
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')

table_rows = soup.find_all('tr', id=lambda x: x and x.startswith('pitching_gamelogs'))

if table_rows:

    data = []

    for row in table_rows:

        row_data = {}

        th = row.find('th')
        
        row_num = th.text
        
        row_data['game_num'] = row_num

        for td in row.find_all('td'):

            stat_type = td['data-stat']

            if stat_type == 'date_game':
                stat_value = td['csk'][:10]
            else:
                stat_value = td.text

            row_data[stat_type] = stat_value

        if row_data:

            game_day = int(row_data['date_game'][8:10])
            game_month = int(row_data['date_game'][5:7])

            if game_month < month:
                    data.append(row_data)
            elif game_month == month:
                if game_day <= day:
                     data.append(row_data)
    
    if data:

        sp_df = pd.DataFrame(data)

        sp_df.rename(columns={
                'earned_run_avg': 'ERA',
                'batters_faced': 'BF',
                'pitches': 'P',
                'strikes_total': 'S',
            }, inplace=True)

        sp_all_columns = sp_df.columns.tolist()

        sp_columns_to_keep = ['game_num', 'date_game', 'opp_ID', 'IP', 'H', 'R', 'ER', 'BB', 'SO', 'HR', 'HBP', 'ERA', 'BF', 'P', 'S', 'AB', '2B', '3B']

        sp_columns_to_drop = [column for column in sp_all_columns if column not in sp_columns_to_keep]

        sp_column_df = sp_df.drop(sp_columns_to_drop, axis=1)

        sp_columns_to_convert_int = ['game_num', 'H', 'R', 'ER', 'BB', 'SO', 'HR', 'HBP', 'BF', 'P', 'S', 'AB', '2B', '3B']

        sp_columns_to_convert_float = ['IP', 'ERA']

        for column in sp_columns_to_convert_int:
            sp_column_df[column] = sp_column_df[column].astype(int)

        for column in sp_columns_to_convert_float:
            sp_column_df[column] = sp_column_df[column].astype(float)
        
        #print(sp_column_df)

        sp_column_df['WHIP'] = sp_column_df.apply(lambda row: ((row['H'] + row['BB']) / row['IP']) if row['IP'] > 0.0 else (row['H'] + row['BB']), axis=1)

        sp_column_df['TB'] = sp_column_df.apply(lambda row: (row['H'] - row['2B'] - row['3B'] - row['HR']) + 2*row['2B'] + 3*row['3B'] + 4*row['HR'], axis=1)

        sp_column_df['IP'] = sp_column_df['IP'].apply(lambda ip: int(ip) + (1/3) if round(ip % 1, 1) == 0.1 else int(ip) + (2/3) if round(ip % 1, 1) == 0.2 else ip)

        sp_column_df['TO'] = sp_column_df.apply(lambda row: row['IP']*3, axis=1)

        sp_column_df['S%'] = sp_column_df.apply(lambda row: row['S'] / row['P'], axis=1)

        sp_column_df['BAA'] = sp_column_df.apply(lambda row: row['H'] / row['AB'], axis=1)

        sp_column_order = ['game_num', 'date_game', 'opp_ID', 'IP', 'P', 'S', 'S%', 'BF', 'AB', 'H', '2B', '3B', 'R', 'ER', 'HR', 'TB', 'BB', 'SO', 'TO', 'HBP', 'ERA', 'WHIP', 'BAA']
                        
        sp_complete_df = sp_column_df[sp_column_order]
    
    else:
        sp_complete_df = None

else:
    sp_complete_df = None

print(sp_complete_df)

    game_num   date_game opp_ID        IP    P   S        S%  BF  AB  H  2B  \
0          1  2024-03-30    SFG  4.666667   85  51  0.600000  19  15  2   2   
1          2  2024-04-05    SFG  6.000000  102  64  0.627451  24  22  4   2   
2          3  2024-04-10    CHC  6.000000  100  62  0.620000  23  21  2   1   
3          4  2024-04-16    MIL  6.000000  110  64  0.581818  24  19  2   1   
4          5  2024-04-22    COL  7.000000   90  59  0.655556  22  20  1   1   
5          6  2024-04-27    PHI  6.000000   91  54  0.593407  27  24  6   0   
6          7  2024-05-03    ARI  6.666667  102  65  0.637255  22  22  3   2   
7          8  2024-05-08    CHC  7.000000  113  69  0.610619  24  21  1   0   
8          9  2024-05-14    COL  5.666667   98  64  0.653061  23  21  5   1   
9         10  2024-05-20    ATL  4.000000   90  57  0.633333  25  22  9   0   
10        11  2024-05-25    NYY  6.666667  107  74  0.691589  28  27  8   1   
11        12  2024-05-31    KCR  5.333333   98  61  

Pitcher Season Stats Up to Desired Date

In [34]:
if sp_complete_df is not None:

    sp_season_stats = {}

    sp_game_data = sp_complete_df.iloc[-1]

    sp_games_started = sp_game_data['game_num']

    print(sp_games_started)

    sp_season_stats['sp_games_started'] = sp_games_started
    sp_season_stats['sp_ip_per_game'] = sp_complete_df['IP'].mean()
    sp_season_stats['sp_p_per_game'] = sp_complete_df['P'].mean()
    sp_season_stats['sp_s_per_game'] = sp_complete_df['S'].mean()
    sp_season_stats['sp_s%'] = sp_complete_df['S'].sum() / sp_complete_df['P'].sum()
    sp_season_stats['sp_bf_per_game'] = sp_complete_df['BF'].mean()
    sp_season_stats['sp_ab_per_game'] = sp_complete_df['AB'].mean()
    sp_season_stats['sp_h_per_game'] = sp_complete_df['H'].mean()
    sp_season_stats['sp_r_per_game'] = sp_complete_df['R'].mean()
    sp_season_stats['sp_er_per_game'] = sp_complete_df['ER'].mean()
    sp_season_stats['sp_hr_per_game'] = sp_complete_df['HR'].mean()
    sp_season_stats['sp_tb_per_game'] = sp_complete_df['TB'].mean()
    sp_season_stats['sp_bb_per_game'] = sp_complete_df['BB'].mean()
    sp_season_stats['sp_k_per_game'] = sp_complete_df['SO'].mean()
    sp_season_stats['sp_k%'] = sp_complete_df['SO'].sum() / sp_complete_df['BF'].sum()
    sp_season_stats['sp_to_per_game'] = sp_complete_df['TO'].mean()
    sp_season_stats['sp_era'] = sp_game_data['ERA']
    sp_season_stats['sp_whip_per_game'] = sp_complete_df['WHIP'].mean()
    sp_season_stats['sp_whip_szn'] = ((sp_complete_df['H'].sum() + sp_complete_df['BB'].sum() ) / sp_complete_df['IP'].sum())
    sp_season_stats['sp_baa'] = sp_complete_df['H'].sum() / sp_complete_df['AB'].sum()

else:
    sp_games_started = 0

    sp_season_stats = {'sp_games_started': 0, 
                       'sp_ip_per_game': None, 
                       'sp_p_per_game': None, 
                       'sp_s_per_game': None, 
                       'sp_s%': None, 
                       'sp_bf_per_game': None, 
                       'sp_ab_per_game': None, 
                       'sp_h_per_game': None, 
                       'sp_r_per_game': None, 
                       'sp_er_per_game': None, 
                       'sp_hr_per_game': None, 
                       'sp_tb_per_game': None, 
                       'sp_bb_per_game': None, 
                       'sp_k_per_game': None, 
                       'sp_k%': None, 
                       'sp_to_per_game': None, 
                       'sp_era': None, 
                       'sp_whip_per_game': None, 
                       'sp_whip_szn': None, 
                       'sp_baa': None
                       }

sp_season_stats_df = pd.DataFrame([sp_season_stats])

print(sp_season_stats_df)

22
   sp_games_started  sp_ip_per_game  sp_p_per_game  sp_s_per_game     sp_s%  \
0                22        5.954545      99.818182      63.227273  0.633424   

   sp_bf_per_game  sp_ab_per_game  sp_h_per_game  sp_r_per_game  \
0            23.5       21.181818       4.045455       2.454545   

   sp_er_per_game  sp_hr_per_game  sp_tb_per_game  sp_bb_per_game  \
0        2.318182        0.681818        7.227273        1.818182   

   sp_k_per_game     sp_k%  sp_to_per_game  sp_era  sp_whip_per_game  \
0       7.636364  0.324952       17.863636     3.5          1.182781   

   sp_whip_szn    sp_baa  
0     0.984733  0.190987  


Format Batter vs Starting Pitcher URL :P

In [35]:
formatted_batter_name = player_name.replace(' ', '%20')

team_schedule = api.schedule(start_date=season_start_date, end_date=date, team=team_id, opponent=opp_id)

if len(team_schedule) > 0:

    games_vs_opp = []
    games_vs_sp = []
    games_vs_sp_pks = []

    for game in team_schedule:
        if game['game_type'] != 'S' and game['status'] != 'Postponed':
            for log in api.get('person_stats', {'personId': player_id, 'gamePk': game['game_id']})['stats'][0]['splits']:
                if log.get('group') == 'hitting' and log['stat']:
                    games_vs_opp.append(game['game_id'])
                    if game['home_id'] == opp_id:
                        if game['home_probable_pitcher'] == sPitcher_name:
                            games_vs_sp.append(game['game_date'])
                            games_vs_sp_pks.append(game['game_id'])
                    elif game['away_id'] == opp_id:
                        if game['away_probable_pitcher'] == sPitcher_name:
                            games_vs_sp.append(game['game_date'])
                            games_vs_sp_pks.append(game['game_id'])
    
    num_games_vs_sp = len(games_vs_sp)
    num_games_vs_opp = len(games_vs_opp)

    game_strings = [f'%22{game_vs_opp}%22' for game_vs_opp in games_vs_opp]

    comma_game_strings =f'[' + ','.join(game_strings) + ']'

    if num_games_vs_sp == 0:
        bvp_url = None
    else:
        bvp_url = f'https://baseballsavant.mlb.com/illustrator?playerId={player_id}&playerType=batter&name={formatted_batter_name}&tabSelection=0&shape=square&shapeName=Square%20(Instagram)&subTitle=&chartType=zoneChart-totalPitches&chartName=Zone%20-%20Total%20Pitches&pitcherThrows=&batterStands=&plateAppearanceResults=[]&pitchTypes=[]&seasonType=[%22R%22]&pitchResults=[]&facingPlayer=[%22{sPitcher_id}%22]&balls=[]&battedBallType=[]&years=[{year}]&strikes=[]&outs=[]&selectedGames=' + comma_game_strings + '&homeTeam=[]&awayTeam=[]&vsTeam=[]&exitVelocityGT=0&exitVelocityLT=125&pitchVelocityGT=0&pitchVelocityLT=105&launchAngleGT=-90&launchAngleLT=90&perspective=catcher&venue=7'


else:
    num_games_vs_sp = 0
    num_games_vs_opp = 0
    games_vs_opp = None
    games_vs_sp = None
    games_vs_sp_pks = None
    bvp_url = None

print(num_games_vs_sp)
print(num_games_vs_opp)

print(bvp_url)
print(games_vs_opp)
print(games_vs_sp)
print(games_vs_sp_pks)


0
8
None
[745444, 746175, 746162, 746161, 746164, 745421, 745422, 745389]
[]
[]


Gather b_vs_opp data

In [36]:
bv_opp_data_dict = {}

if num_games_vs_opp > 0:

    bv_opp_raw_data = []

    for game in games_vs_opp:
        game_vs_opp_log = api.get('person_stats', {'personId': player_id, 'gamePk': game})['stats'][0]['splits']
        game_vs_opp_data = {}
        for group in game_vs_opp_log:
            if group['group'] == 'hitting':
                for s in group['stat']:
                    game_vs_opp_data[s] = group['stat'][s]
        bv_opp_raw_data.append(game_vs_opp_data)
    
    bv_opp_raw_data_df = pd.DataFrame(bv_opp_raw_data)

    bv_opp_raw_data_df.rename(columns={
        'runs': 'bv_opp_r',
        'doubles': '2B',
        'triples': '3B',
        'homeRuns': 'bv_opp_hr',
        'strikeOuts': 'bv_opp_k',
        'baseOnBalls': 'bv_opp_bb',
        'hits': 'bv_opp_h',
        'hitByPitch': 'HBP',
        'atBats': 'bv_opp_ab',
        'plateAppearances': 'bv_opp_pa',
        'totalBases': 'bv_opp_tb',
        'rbi': 'bv_opp_rbi',
        'sacFlies': 'SF'
    }, inplace=True)

    bv_opp_columns = bv_opp_raw_data_df.columns.to_list()

    bv_opp_columns_to_keep = ['bv_opp_pa', 'bv_opp_ab', 'bv_opp_h', 'bv_opp_r', 'bv_opp_rbi', '2B', '3B', 'bv_opp_hr', 'bv_opp_tb', 'bv_opp_bb', 'bv_opp_k', 'HBP', 'SF']

    bv_opp_columns_to_drop = [column for column in bv_opp_columns if column not in bv_opp_columns_to_keep]

    bv_opp_columns_df = bv_opp_raw_data_df.drop(bv_opp_columns_to_drop, axis=1)

    bv_opp_columns_df.astype(int)

    for column in bv_opp_columns_df.columns.to_list():
        bv_opp_data_dict[column] = bv_opp_columns_df[column].mean()

    bv_opp_data_dict['bv_opp_ba'] = bv_opp_data_dict['bv_opp_h'] / bv_opp_data_dict['bv_opp_ab']
    bv_opp_data_dict['bv_opp_obp'] = (bv_opp_data_dict['bv_opp_h'] + bv_opp_data_dict['bv_opp_bb'] + bv_opp_data_dict['HBP']) / (
        bv_opp_data_dict['bv_opp_ab'] + bv_opp_data_dict['bv_opp_bb'] + bv_opp_data_dict['HBP'] + bv_opp_data_dict['SF'])
    bv_opp_data_dict['bv_opp_slg'] = bv_opp_data_dict['bv_opp_tb'] / bv_opp_data_dict['bv_opp_ab']

    for key in ['2B', '3B', 'HBP', 'SF']:
        bv_opp_data_dict.pop(key)
    
    bv_opp_data_dict['bv_opp_games_played'] = num_games_vs_opp

else:
    bv_opp_data_dict = {'bv_opp_games_played': None, 'bv_opp_pa': None, 'bv_opp_ab': None, 'bv_opp_h': None,'bv_opp_r': None, 'bv_opp_rbi': None, 'bv_opp_hr': None, 'bv_opp_tb': None, 'bv_opp_bb': None,
                         'bv_opp_k': None, 'bv_opp_ba': None, 'bv_opp_obp': None, 'bv_opp_slg': None}

bv_opp_data_df = pd.DataFrame([bv_opp_data_dict])

bv_opp_column_order = ['bv_opp_games_played', 'bv_opp_pa', 'bv_opp_ab', 'bv_opp_h', 'bv_opp_r', 'bv_opp_rbi', 'bv_opp_hr', 'bv_opp_tb', 'bv_opp_bb', 'bv_opp_k', 'bv_opp_ba', 'bv_opp_obp', 'bv_opp_slg']

bv_opp_df = bv_opp_data_df[bv_opp_column_order]

print(bv_opp_df)

   bv_opp_games_played  bv_opp_pa  bv_opp_ab  bv_opp_h  bv_opp_r  bv_opp_rbi  \
0                    8       4.75        4.0     1.125       0.5         0.5   

   bv_opp_hr  bv_opp_tb  bv_opp_bb  bv_opp_k  bv_opp_ba  bv_opp_obp  \
0      0.125      1.875        0.5     0.625    0.28125    0.342105   

   bv_opp_slg  
0     0.46875  


Gather bv_opp_sp_stats

In [37]:
bv_opp_sp_data_dict = {}

if num_games_vs_sp > 0:

    bv_opp_sp_raw_data = []

    for game in games_vs_sp_pks:
        game_vs_opp_sp_log = api.get('person_stats', {'personId': player_id, 'gamePk': game})['stats'][0]['splits']
        game_vs_opp_sp_data = {}
        for group in game_vs_opp_sp_log:
            if group['group'] == 'hitting':
                for s in group['stat']:
                    game_vs_opp_sp_data[s] = group['stat'][s]
        bv_opp_sp_raw_data.append(game_vs_opp_sp_data)
    
    bv_opp_sp_raw_data_df = pd.DataFrame(bv_opp_sp_raw_data)

    bv_opp_sp_raw_data_df.rename(columns={
        'runs': 'bv_opp_sp_r',
        'doubles': '2B',
        'triples': '3B',
        'homeRuns': 'bv_opp_sp_hr',
        'strikeOuts': 'bv_opp_sp_k',
        'baseOnBalls': 'bv_opp_sp_bb',
        'hits': 'bv_opp_sp_h',
        'hitByPitch': 'HBP',
        'atBats': 'bv_opp_sp_ab',
        'plateAppearances': 'bv_opp_sp_pa',
        'totalBases': 'bv_opp_sp_tb',
        'rbi': 'bv_opp_sp_rbi',
        'sacFlies': 'SF'
    }, inplace=True)

    bv_opp_sp_columns = bv_opp_sp_raw_data_df.columns.to_list()

    bv_opp_sp_columns_to_keep = ['bv_opp_sp_pa', 'bv_opp_sp_ab', 'bv_opp_sp_h', 'bv_opp_sp_r', 'bv_opp_sp_rbi', '2B', '3B', 
                                 'bv_opp_sp_hr', 'bv_opp_sp_tb', 'bv_opp_sp_bb', 'bv_opp_sp_k', 'HBP', 'SF']

    bv_opp_sp_columns_to_drop = [column for column in bv_opp_sp_columns if column not in bv_opp_sp_columns_to_keep]

    bv_opp_sp_columns_df = bv_opp_sp_raw_data_df.drop(bv_opp_sp_columns_to_drop, axis=1)

    bv_opp_sp_columns_df.astype(int)

    for column in bv_opp_sp_columns_df.columns.to_list():
        bv_opp_sp_data_dict[column] = bv_opp_sp_columns_df[column].mean()
    
    bv_opp_sp_data_dict['bv_opp_sp_ba'] = bv_opp_sp_data_dict['bv_opp_sp_h'] / bv_opp_sp_data_dict['bv_opp_sp_ab']
    bv_opp_sp_data_dict['bv_opp_sp_obp'] = (bv_opp_sp_data_dict['bv_opp_sp_h'] + bv_opp_sp_data_dict['bv_opp_sp_bb'] + bv_opp_sp_data_dict['HBP']) / (
        bv_opp_sp_data_dict['bv_opp_sp_ab'] + bv_opp_sp_data_dict['bv_opp_sp_bb'] + bv_opp_sp_data_dict['HBP'] + bv_opp_sp_data_dict['SF'])
    bv_opp_sp_data_dict['bv_opp_sp_slg'] = bv_opp_sp_data_dict['bv_opp_sp_tb'] / bv_opp_sp_data_dict['bv_opp_sp_ab']

    for key in ['2B', '3B', 'HBP', 'SF']:
        bv_opp_sp_data_dict.pop(key)

else:
    bv_opp_sp_data_dict = {'bv_opp_sp_pa': None, 'bv_opp_sp_ab': None, 'bv_opp_sp_h': None,'bv_opp_sp_r': None, 'bv_opp_sp_rbi': None, 'bv_opp_sp_hr': None, 'bv_opp_sp_tb': None, 
                           'bv_opp_sp_bb': None, 'bv_opp_sp_k': None, 'bv_opp_sp_ba': None, 'bv_opp_sp_obp': None, 'bv_opp_sp_slg': None}

bv_opp_sp_stats_df = pd.DataFrame([bv_opp_sp_data_dict])

bv_opp_sp_column_order = ['bv_opp_sp_pa', 'bv_opp_sp_ab', 'bv_opp_sp_h', 'bv_opp_sp_r', 'bv_opp_sp_rbi', 'bv_opp_sp_hr', 'bv_opp_sp_tb', 
                          'bv_opp_sp_bb', 'bv_opp_sp_k', 'bv_opp_sp_ba', 'bv_opp_sp_obp', 'bv_opp_sp_slg']

bv_opp_sp_df = bv_opp_sp_stats_df[bv_opp_sp_column_order]

print(bv_opp_sp_df)

  bv_opp_sp_pa bv_opp_sp_ab bv_opp_sp_h bv_opp_sp_r bv_opp_sp_rbi  \
0         None         None        None        None          None   

  bv_opp_sp_hr bv_opp_sp_tb bv_opp_sp_bb bv_opp_sp_k bv_opp_sp_ba  \
0         None         None         None        None         None   

  bv_opp_sp_obp bv_opp_sp_slg  
0          None          None  


Gather sp_vs_team stats

In [38]:
sp_vs_team_dict = {'sp_vs_team_ip': None, 
                    'sp_vs_team_p': None, 
                    'sp_vs_team_s': None, 
                    'sp_vs_team_s%': None, 
                    'sp_vs_team_bf': None, 
                    'sp_vs_team_ab': None, 
                    'sp_vs_team_h': None, 
                    'sp_vs_team_r': None, 
                    'sp_vs_team_er': None, 
                    'sp_vs_team_hr': None, 
                    'sp_vs_team_tb': None, 
                    'sp_vs_team_bb': None, 
                    'sp_vs_team_k': None, 
                    'sp_vs_team_k%': None, 
                    'sp_vs_team_to': None, 
                    'sp_vs_team_era': None, 
                    'sp_vs_team_whip': None, 
                    'sp_vs_team_baa': None
                    }

if sp_games_started > 0:

    sp_vs_team_logs = []

    for j in range(len(sp_complete_df)):
        game_row = sp_complete_df.iloc[j]

        if api.lookup_team(game_row['opp_ID'].lower()[:2])[0]['id'] == team_id:
            sp_vs_team_logs.append(game_row)

    if len(sp_vs_team_logs) > 0:
        sp_vs_team_raw_df = pd.DataFrame(sp_vs_team_logs)

        sp_vs_team_dict['sp_vs_team_ip'] = sp_vs_team_raw_df['IP'].mean()
        sp_vs_team_dict['sp_vs_team_p'] = sp_vs_team_raw_df['P'].mean()
        sp_vs_team_dict['sp_vs_team_s'] = sp_vs_team_raw_df['S'].mean()
        sp_vs_team_dict['sp_vs_team_s%'] = sp_vs_team_raw_df['S'].sum() / sp_complete_df['P'].sum()
        sp_vs_team_dict['sp_vs_team_bf'] = sp_vs_team_raw_df['BF'].mean()
        sp_vs_team_dict['sp_vs_team_ab'] = sp_vs_team_raw_df['AB'].mean()
        sp_vs_team_dict['sp_vs_team_h'] = sp_vs_team_raw_df['H'].mean()
        sp_vs_team_dict['sp_vs_team_r'] = sp_vs_team_raw_df['R'].mean()
        sp_vs_team_dict['sp_vs_team_er'] = sp_vs_team_raw_df['ER'].mean()
        sp_vs_team_dict['sp_vs_team_hr'] = sp_vs_team_raw_df['HR'].mean()
        sp_vs_team_dict['sp_vs_team_tb'] = sp_vs_team_raw_df['TB'].mean()
        sp_vs_team_dict['sp_vs_team_bb'] = sp_vs_team_raw_df['BB'].mean()
        sp_vs_team_dict['sp_vs_team_k'] = sp_vs_team_raw_df['SO'].mean()
        sp_vs_team_dict['sp_vs_team_k%'] = sp_vs_team_raw_df['SO'].sum() / sp_vs_team_raw_df['BF'].sum()
        sp_vs_team_dict['sp_vs_team_to'] = sp_vs_team_raw_df['TO'].mean()
        sp_vs_team_dict['sp_vs_team_era'] = sp_vs_team_raw_df.iloc[-1]['ERA']
        sp_vs_team_dict['sp_vs_team_whip'] = ((sp_vs_team_raw_df['H'].sum() + sp_vs_team_raw_df['BB'].sum() ) / sp_vs_team_raw_df['IP'].sum())
        sp_vs_team_dict['sp_vs_team_baa'] = sp_vs_team_raw_df['H'].sum() / sp_vs_team_raw_df['AB'].sum()

sp_vs_team_stats_df = pd.DataFrame([sp_vs_team_dict])

print(sp_vs_team_stats_df)


  sp_vs_team_ip sp_vs_team_p sp_vs_team_s sp_vs_team_s% sp_vs_team_bf  \
0          None         None         None          None          None   

  sp_vs_team_ab sp_vs_team_h sp_vs_team_r sp_vs_team_er sp_vs_team_hr  \
0          None         None         None          None          None   

  sp_vs_team_tb sp_vs_team_bb sp_vs_team_k sp_vs_team_k% sp_vs_team_to  \
0          None          None         None          None          None   

  sp_vs_team_era sp_vs_team_whip sp_vs_team_baa  
0           None            None           None  


Gather opp_stats

In [39]:
search_url = f'https://www.baseball-reference.com/teams/tgl.cgi?team={opp_abv}&t=p&year={year}#team_pitching_gamelogs'

response = requests.get(search_url)
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')

table_rows = soup.find_all('tr', id=lambda x: x and x.startswith('team_pitching_gamelogs'))

if table_rows:

    data = []

    for row in table_rows:

        row_data = {}

        for td in row.find_all('td'):

            stat_type = td['data-stat']

            if stat_type == 'date_game':
                stat_value = td['csk'][:10]
            else:
                stat_value = td.text

            row_data[stat_type] = stat_value

        if row_data:

            game_day = int(row_data['date_game'][8:10])
            game_month = int(row_data['date_game'][5:7])

            if game_month < month:
                    data.append(row_data)
            elif game_month == month:
                if game_day <= day:
                     data.append(row_data)

    if data:
        opp_raw_df = pd.DataFrame(data)

        opp_raw_df.rename(columns={
             'team_game_num': 'opp_game_num',
             'earned_run_avg': 'ERA',
             'batters_faced': 'BF',
             'pitches': 'P',
             'strikes_total': 'S',
             'pitchers_number': 'num_pitchers'
        }, inplace=True)

        opp_all_columns = opp_raw_df.columns.tolist()

        opp_columns_to_keep = ['opp_game_num', 'IP', 'H', 'R', 'ER', 'BB', 'SO', 'HR', 'HBP', 'ERA', 'BF', 'P', 'S', 'AB', '2B', '3B', 'num_pitchers']

        opp_columns_to_drop = [column for column in opp_all_columns if column not in opp_columns_to_keep]

        opp_column_df = opp_raw_df.drop(opp_columns_to_drop, axis=1)

        for column in opp_column_df.columns.tolist():
            if column in ['IP', 'ERA']:
                opp_column_df[column] = opp_column_df[column].astype(float)
            else:
                opp_column_df[column] = opp_column_df[column].astype(int)
        
        opp_column_df['WHIP'] = opp_column_df.apply(lambda row: ((row['H'] + row['BB']) / row['IP']) if row['IP'] > 0.0 else (row['H'] + row['BB']), axis=1)

        opp_column_df['TB'] = opp_column_df.apply(lambda row: (row['H'] - row['2B'] - row['3B'] - row['HR']) + 2*row['2B'] + 3*row['3B'] + 4*row['HR'], axis=1)

        opp_column_df['TO'] = opp_column_df.apply(lambda row: row['IP']*3, axis=1)

        opp_column_df['S%'] = opp_column_df.apply(lambda row: row['S'] / row['P'], axis=1)

    else:
        opp_column_df = None

else:
    opp_column_df = None

print(opp_column_df)

     opp_game_num    IP   H   R  ER  BB  SO  HR  HBP   ERA  BF    P    S  AB  \
0               1   9.0   7   5   3   9   7   0    1  3.00  45  173  100  33   
1               2   9.0  16  11  11   6   9   1    1  7.00  50  191  119  42   
2               3   9.0   9   4   4   1  10   1    0  6.00  35  131   92  32   
3               4   9.0  13   8   8   2   7   2    2  6.50  42  156  106  38   
4               5   9.0   9   9   8   3  10   2    0  6.80  40  164  105  35   
5               6   9.0   5   4   4   8   9   1    1  6.33  40  148   83  31   
6               7   9.0  14   6   6   3  13   2    1  6.29  45  173  115  41   
7               8   9.0   6   5   4   4   7   1    0  6.00  37  124   79  29   
8               9   9.0   6   2   1   4  11   0    4  5.44  36  155   88  28   
9              10   8.1   5   3   3   2   8   0    1  5.24  33  141   89  30   
10             11   9.0   4   0   0   3   5   0    0  4.76  34  123   80  31   
11             12   8.0   6   3   0   2 

In [40]:
opp_stats_dict = {'opp_num_pitchers_per_game': None, 'opp_p_per_game': None, 'opp_s_per_game': None, 'opp_s%': None, 'opp_bf_per_game': None, 
                  'opp_ab_per_game': None, 'opp_h_per_game': None, 'opp_r_per_game': None, 'opp_er_per_game': None, 'opp_hr_per_game': None, 
                  'opp_tb_per_game': None, 'opp_bb_per_game': None, 'opp_k_per_game': None, 'opp_k%': None, 'opp_to_per_game': None, 
                  'opp_era': None, 'opp_whip': None, 'opp_baa': None
                  }

if opp_column_df is not None:
    
    opp_stats_dict['opp_num_pitchers_per_game'] = opp_column_df['num_pitchers'].mean()
    opp_stats_dict['opp_p_per_game'] = opp_column_df['P'].mean()
    opp_stats_dict['opp_s_per_game'] = opp_column_df['S'].mean()
    opp_stats_dict['opp_s%'] = opp_column_df['S'].sum() / opp_column_df['P'].sum()
    opp_stats_dict['opp_bf_per_game'] = opp_column_df['BF'].mean()
    opp_stats_dict['opp_ab_per_game'] = opp_column_df['AB'].mean()
    opp_stats_dict['opp_h_per_game'] = opp_column_df['H'].mean()
    opp_stats_dict['opp_r_per_game'] = opp_column_df['R'].mean()
    opp_stats_dict['opp_er_per_game'] = opp_column_df['ER'].mean()
    opp_stats_dict['opp_hr_per_game'] = opp_column_df['HR'].mean()
    opp_stats_dict['opp_tb_per_game'] = opp_column_df['TB'].mean()
    opp_stats_dict['opp_bb_per_game'] = opp_column_df['BB'].mean()
    opp_stats_dict['opp_k_per_game'] = opp_column_df['SO'].mean()
    opp_stats_dict['opp_k%'] = opp_column_df['SO'].sum() / opp_column_df['BF'].sum()
    opp_stats_dict['opp_to_per_game'] = opp_column_df['TO'].mean()
    opp_stats_dict['opp_era'] = opp_column_df.iloc[-1]['ERA']
    opp_stats_dict['opp_whip'] = (opp_column_df['H'].sum() + opp_column_df['BB'].sum()) / opp_column_df['IP'].sum()
    opp_stats_dict['opp_baa'] = opp_column_df['H'].sum() / opp_column_df['AB'].sum()

opp_stats_df = pd.DataFrame([opp_stats_dict])

print(opp_stats_df)

   opp_num_pitchers_per_game  opp_p_per_game  opp_s_per_game    opp_s%  \
0                   4.266055      146.385321       94.311927  0.644272   

   opp_bf_per_game  opp_ab_per_game  opp_h_per_game  opp_r_per_game  \
0        37.100917        33.412844        8.100917        4.293578   

   opp_er_per_game  opp_hr_per_game  opp_tb_per_game  opp_bb_per_game  \
0              4.0         1.082569        13.238532          2.87156   

   opp_k_per_game    opp_k%  opp_to_per_game  opp_era  opp_whip   opp_baa  
0        8.798165  0.237141        26.537615     4.07  1.240407  0.242449  


Gather team_v_opp_stats

In [41]:
team_vs_opp_schedule = api.schedule(start_date=season_start_date, end_date= date, team=team_id, opponent=opp_id)

team_vs_opp_games = []
team_vs_opp_game_pks = []

if len(team_vs_opp_schedule) > 0:

    for game in team_vs_opp_schedule:
        if game['game_type'] != 'S' and game['status'] != 'Postponed':
            team_vs_opp_games.append(game['game_date'])
            team_vs_opp_game_pks.append(game['game_id'])
    
team_vs_opp_games_played = len(team_vs_opp_games)

team_vs_opp_data_dict = {'team_vs_opp_games_played': None, 'team_vs_opp_num_batters': None, 'team_vs_opp_pa': None, 'team_vs_opp_ab': None, 
                         'team_vs_opp_h': None,'team_vs_opp_r': None, 'team_vs_opp_rbi': None, 'team_vs_opp_hr': None, 'team_vs_opp_tb': None, 
                         'team_vs_opp_bb': None,'team_vs_opp_k': None, 'team_vs_opp_ba': None, 'team_vs_opp_obp': None, 'team_vs_opp_slg': None}

if team_vs_opp_games_played > 0:

    response = requests.get(f'https://www.baseball-reference.com/teams/tgl.cgi?team={team_abv}&t=b&year={year}#team_batting_gamelogs')
    response.raise_for_status()

    soup = BeautifulSoup(response.content, 'html.parser')

    table_rows = soup.find_all('tr', id=lambda x: x and x.startswith('team_batting_gamelogs'))

    if table_rows:

        data = []

        for row in table_rows:

            row_data = {}

            for td in row.find_all('td'):

                stat_type = td['data-stat']

                if stat_type == 'date_game':
                    stat_value = td['csk'][:10]
                else:
                    stat_value = td.text

                row_data[stat_type] = stat_value

            if row_data:

                if row_data['date_game'] in team_vs_opp_games:

                    game_day = int(row_data['date_game'][8:10])
                    game_month = int(row_data['date_game'][5:7])

                    if game_month < month:
                            data.append(row_data)
                    elif game_month == month:
                        if game_day <= day:
                            data.append(row_data)
        
        if data:
            team_vs_opp_raw_df = pd.DataFrame(data)

            #print(team_vs_opp_raw_df)

            team_vs_opp_raw_df.rename(columns={
                'batting_avg': 'BA',
                'onbase_perc': 'OBP',
                'slugging_perc': 'SLG',
                'batters_number': 'num_batters'
            }, inplace=True)

            team_vs_opp_columns = team_vs_opp_raw_df.columns.tolist()

            team_vs_opp_columns_to_keep = ['PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'HBP', 'num_batters']

            team_vs_opp_columns_to_drop = [column for column in team_vs_opp_columns if column not in team_vs_opp_columns_to_keep]

            team_vs_opp_columns_df = team_vs_opp_raw_df.drop(team_vs_opp_columns_to_drop, axis=1)

            team_vs_opp_columns_int = ['PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'HBP', 'num_batters']

            team_vs_opp_columns_float = ['BA', 'OBP', 'SLG']

            for column in team_vs_opp_columns_int:
                team_vs_opp_columns_df[column] = team_vs_opp_columns_df[column].astype(int)

            for column in team_vs_opp_columns_float:
                team_vs_opp_columns_df[column] = team_vs_opp_columns_df[column].astype(float)

            team_vs_opp_columns_df['TB'] = team_vs_opp_columns_df.apply(lambda row: (row['H'] - row['2B'] - row['3B'] - row['HR']) + 2*row['2B'] + 3*row['3B'] + 4*row['HR'], axis=1)
            
        else:
            team_vs_opp_columns_df = None
    
    else:
        team_vs_opp_columns_df = None
    
    if team_vs_opp_columns_df is not None:
        
        team_vs_opp_data_dict['team_vs_opp_games_played'] = team_vs_opp_games_played
        team_vs_opp_data_dict['team_vs_opp_num_batters'] = team_vs_opp_columns_df['num_batters'].mean()
        team_vs_opp_data_dict['team_vs_opp_pa'] = team_vs_opp_columns_df['PA'].mean()
        team_vs_opp_data_dict['team_vs_opp_ab'] = team_vs_opp_columns_df['AB'].mean()
        team_vs_opp_data_dict['team_vs_opp_h'] = team_vs_opp_columns_df['H'].mean()
        team_vs_opp_data_dict['team_vs_opp_r'] = team_vs_opp_columns_df['R'].mean()
        team_vs_opp_data_dict['team_vs_opp_rbi'] = team_vs_opp_columns_df['RBI'].mean()
        team_vs_opp_data_dict['team_vs_opp_hr'] = team_vs_opp_columns_df['HR'].mean()
        team_vs_opp_data_dict['team_vs_opp_tb'] = team_vs_opp_columns_df['TB'].mean()
        team_vs_opp_data_dict['team_vs_opp_bb'] = team_vs_opp_columns_df['BB'].mean()
        team_vs_opp_data_dict['team_vs_opp_k'] = team_vs_opp_columns_df['SO'].mean()
        team_vs_opp_data_dict['team_vs_opp_ba'] = team_vs_opp_columns_df['H'].sum() / team_vs_opp_columns_df['AB'].sum()
        team_vs_opp_data_dict['team_vs_opp_obp'] = (team_vs_opp_columns_df['H'].sum() + team_vs_opp_columns_df['BB'].sum() + team_vs_opp_columns_df['HBP'].sum()) / team_vs_opp_columns_df['PA'].sum()
        team_vs_opp_data_dict['team_vs_opp_slg'] = team_vs_opp_columns_df['TB'].sum() / team_vs_opp_columns_df['AB'].sum()

team_vs_opp_df = pd.DataFrame([team_vs_opp_data_dict])

print(team_vs_opp_df)


   team_vs_opp_games_played  team_vs_opp_num_batters  team_vs_opp_pa  \
0                         9                 9.666667       40.222222   

   team_vs_opp_ab  team_vs_opp_h  team_vs_opp_r  team_vs_opp_rbi  \
0       34.444444       7.111111       4.666667         4.555556   

   team_vs_opp_hr  team_vs_opp_tb  team_vs_opp_bb  team_vs_opp_k  \
0             1.0       11.222222        4.666667       8.666667   

   team_vs_opp_ba  team_vs_opp_obp  team_vs_opp_slg  
0        0.206452         0.301105         0.325806  


Series Data

In [42]:
series_data_dict = {'series_games_played': None, 'series_pa': None, 'series_ab': None, 'series_h': None, 'series_r': None, 
                    'series_rbi': None, 'series_hr': None, 'series_tb': None, 'series_bb': None, 'series_k': None, 
                    'series_ba': None, 'series_obp': None, 'series_slg': None
                    }

#team_vs_opp_games = ['2024-05-25', '2024-05-26', '2024-06-14', '2024-06-15', '2024-06-16', '2024-07-12', '2024-07-13', '2024-07-13', '2024-07-14', '2024-08-01']
series_status = False

if games_vs_opp is not None:
    i = 0
    while i < len(team_vs_opp_game_pks):
        if team_vs_opp_game_pks[i] not in games_vs_opp:
            team_vs_opp_games.pop(i)
        i += 1

    print(team_vs_opp_games)

    if len(games_vs_opp) >= 3:
        series_games = team_vs_opp_games[len(games_vs_opp) - 3:]

    elif len(games_vs_opp) == 2:
        series_games = team_vs_opp_games[len(games_vs_opp) - 2:]

    elif len(games_vs_opp) == 1:
        series_games = [team_vs_opp_games[0]]

    else:
        series_games = None

    print(series_games)

    if series_games is not None:
        series_games_data = []
        next_game_num = games_played + 1
        for k in range(len(correct_columns_df)):
            game = correct_columns_df.iloc[k]
            if game['date_game'] in series_games:
                if game['game_num'] in range(next_game_num - 3, next_game_num):
                    series_games_data.append(game)
        
        series_games_played = len(series_games_data)

        if series_games_played > 0:
            series_status = True

            series_games_df = pd.DataFrame(series_games_data)

            series_data_dict['series_games_played'] = series_games_played
            series_data_dict['series_pa'] = series_games_df['PA'].mean()
            series_data_dict['series_ab'] = series_games_df['AB'].mean()
            series_data_dict['series_h'] = series_games_df['H'].mean()
            series_data_dict['series_r'] = series_games_df['R'].mean()
            series_data_dict['series_rbi'] = series_games_df['RBI'].mean()
            series_data_dict['series_hr'] = series_games_df['HR'].mean()
            series_data_dict['series_tb'] = series_games_df['TB'].mean()
            series_data_dict['series_bb'] = series_games_df['BB'].mean()
            series_data_dict['series_k'] = series_games_df['SO'].mean()
            series_data_dict['series_ba'] = series_games_df['H'].sum() / series_games_df['AB'].sum()
            series_data_dict['series_obp'] = (series_games_df['H'].sum() + series_games_df['BB'].sum() + series_games_df['HBP'].sum()) / series_games_df['PA'].sum()
            series_data_dict['series_slg'] = series_games_df['TB'].sum() / series_games_df['AB'].sum()


series_stats_df = pd.DataFrame([series_data_dict])

print(series_status)
print(series_stats_df)

['2024-03-20', '2024-03-21', '2024-04-12', '2024-04-13', '2024-04-14', '2024-05-10', '2024-05-11', '2024-07-30']
['2024-05-10', '2024-05-11', '2024-07-30']
True
   series_games_played  series_pa  series_ab  series_h  series_r  series_rbi  \
0                    1        5.0        4.0       0.0       1.0         0.0   

   series_hr  series_tb  series_bb  series_k  series_ba  series_obp  \
0        0.0        0.0        1.0       2.0        0.0         0.2   

   series_slg  
0         0.0  


INITIALIZING CHROMEDRIVER

In [43]:
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-extensions")
options.add_argument("--disable-images")
prefs = {
    'profile.default_content_setting_values': {
        'images': 2,
        'javascript': 2,
        'stylesheet': 2
    }
}
options.add_experimental_option('prefs', prefs)

#service = Service(chrome_driver_path)
#driver = webdriver.Chrome(service=service, options=options)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

Gather Raw BVP Data

In [44]:
if bvp_url is not None:
    bvp_raw_data = {}

    #service = Service(chrome_driver_path)
    #driver = webdriver.Chrome(service=service, options=options)
    
    #with webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) as driver:
    try:
        driver.get(bvp_url)

        driver.implicitly_wait(10)

        stats_boxes = driver.find_elements(By.CLASS_NAME, 'stats-box')

        for stats_box in stats_boxes:
            stat_type = stats_box.find_element(By.CLASS_NAME, 'stats-title')
            stat_value = stats_box.find_element(By.CLASS_NAME, 'stats-value')
            bvp_raw_data[stat_type.text] = stat_value.text
        
        for key in bvp_raw_data.keys():
            bvp_raw_data[key] = float(bvp_raw_data[key])
    
    except Exception as e:
        bvp_raw_data = None

    #finally:
        #driver.close()
        #driver.quit()
        #kill_chrome_processes()

else:
    bvp_raw_data = None

print(bvp_raw_data)

None


Manipulate bvp data

TO DO: Remove normalization to 9 innings and keep as total per game.
Consider: switching to per plate appearance
Consider: instead of just looking at bvp, look at entire game where that pitcher was sp
            - record stats per plate appearance against starting pitcher
            - record stats per plate appearance against entire bullpen
            - most importantly, determine avg of plate appearances against sp per game and stats associated with those plate appearances
            - big idea: want to determine how many plate appearances the batter will get vs this sp, and then determine the likelihood of player getting a hit within those plate appearances.
            - Then want to determine how many plate appearances the batter will get after sp, then then determine the likelihood of a player getting a hit within those plate appearances.
            - Combine both of these stats to predict the amount of plate appearances the batter will get per game, and how likely they are to get a hit given the split of those plate appearances 

In [45]:
bvp_data = {'bvp_games_faced': None,
            'bvp_ip_total': None,
            'bvp_ip_per_game': None,
            'bvp_p_total': None,
            'bvp_p_per_game': None,
            'bvp_pa_total': None,
            'bvp_pa_per_game': None,
            'bvp_h_total': None,
            'bvp_h_per_game': None,
            'bvp_hr_total': None,
            'bvp_hr_per_game': None,
            'bvp_tb_total': None,
            'bvp_tb_per_game': None,
            'bvp_k_total': None,
            'bvp_k_per_game': None,
            'bvp_k%': None,
            'bvp_woba': None,
            'bvp_ba': None,
            }

if bvp_raw_data is not None:
    bvp_total_data = {}
    sp_ip = 0.0

    for d in games_vs_sp:
        sp_ip += float(sp_column_df[sp_column_df['date_game'] == d]['IP'].iloc[0])

    sp_ip_per_bvp = round(sp_ip / num_games_vs_sp, 3)
    bvp_total_data['bvp_ip_total'] = sp_ip
    bvp_total_data['bvp_ip_per_game'] = sp_ip_per_bvp

    bvp_total_data['bvp_p_total'] = bvp_raw_data['Pitches']
    bvp_total_data['bvp_p_per_game'] = bvp_raw_data['Pitches'] / num_games_vs_sp
    bvp_total_data['bvp_pa_total'] = bvp_raw_data['PA']
    bvp_total_data['bvp_pa_per_game'] = bvp_raw_data['PA'] / num_games_vs_sp
    bvp_total_data['bvp_h_total'] = bvp_raw_data['H']
    bvp_total_data['bvp_h_per_game'] = bvp_raw_data['H'] / num_games_vs_sp
    bvp_total_data['bvp_1B_adj'] = bvp_raw_data['1B']
    bvp_total_data['bvp_2B_adj'] = bvp_raw_data['2B']
    bvp_total_data['bvp_3B_adj'] = bvp_raw_data['3B']
    bvp_total_data['bvp_hr_total'] = bvp_raw_data['HR']
    bvp_total_data['bvp_hr_per_game'] = bvp_raw_data['HR'] / num_games_vs_sp
    bvp_total_data['bvp_k_total'] = bvp_raw_data['SO']
    bvp_total_data['bvp_k_per_game'] = bvp_raw_data['SO'] / num_games_vs_sp
    
    bvp_total_data['bvp_tb_total'] = (bvp_total_data['bvp_1B_adj'] + 2 * bvp_total_data['bvp_2B_adj']
                                                  + 3 * bvp_total_data['bvp_3B_adj'] + 4 * bvp_total_data['bvp_hr_per_game'])
    bvp_total_data['bvp_tb_per_game'] = bvp_total_data['bvp_tb_total'] / num_games_vs_sp

    bvp_data['bvp_ba'] = bvp_raw_data['AVG']
    bvp_data['bvp_k%'] = bvp_raw_data['K%']
    bvp_data['bvp_woba'] = bvp_raw_data['wOBA']

    for i in ['bvp_1B_adj', 'bvp_2B_adj', 'bvp_3B_adj']:
        bvp_total_data.pop(i)
    
    for total in bvp_total_data.keys():
        bvp_data[total] = bvp_total_data[total]
    
    bvp_data['bvp_games_faced'] = num_games_vs_sp
    
    bvp_data_df = pd.DataFrame([bvp_data]).astype(float)

else:
    bvp_data_df = pd.DataFrame([bvp_data])

print(bvp_data_df)
        

  bvp_games_faced bvp_ip_total bvp_ip_per_game bvp_p_total bvp_p_per_game  \
0            None         None            None        None           None   

  bvp_pa_total bvp_pa_per_game bvp_h_total bvp_h_per_game bvp_hr_total  \
0         None            None        None           None         None   

  bvp_hr_per_game bvp_tb_total bvp_tb_per_game bvp_k_total bvp_k_per_game  \
0            None         None            None        None           None   

  bvp_k% bvp_woba bvp_ba  
0   None     None   None  


Fetch Umpire Stats
- If Home Plate Ump announced
    - Return data for only HP ump 
- If Home Plate Ump not yet announced
    - Fetch data for all 4 umps
    - hp_ump stats = avg of 4 umps stats

In [46]:
if len(proj_ump_ids) > 0:

    #service = Service(chrome_driver_path)
    #driver = webdriver.Chrome(service=service, options=options)

    #with webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) as driver:
    try:
        driver.get('https://swishanalytics.com/mlb/mlb-umpire-factors')

        table = driver.find_element(By.CSS_SELECTOR, "div.table-responsive.table-umpires > table")

        rows = table.find_elements(By.CSS_SELECTOR, "tbody tr")

        umpire_data = []

        for row in rows:
            umpire_name = row.find_element(By.CSS_SELECTOR, "td.umps-name").text.strip()
            if umpire_name in proj_umps:
                row_data = [cell.text.strip() for cell in row.find_elements(By.CSS_SELECTOR, "td")]
                umpire_data.append(row_data[3:])
        
        for u in umpire_data:
            i = 0
            while i < len(u):
                if u[i].endswith('%'):
                    u[i] = float(u[i].strip('%')) / 100
                elif u[i].endswith('x'):
                    u[i] = float(u[i].strip('x'))
                else:
                    u[i] = float(u[i])
                i += 1
    
    except Exception as e:
        umpire_data = [[None for _ in range(12)]]
                
    #finally:
        #driver.close()
        #driver.quit()

else:
    umpire_data = [[None for _ in range(12)]]


print(umpire_data)


[[0.223, 0.081, 8.61, 0.246, 0.313, 0.402, 1.0, 0.95, 0.97, 0.99, 0.98, 0.98]]


In [47]:
umpire_data_df = pd.DataFrame(columns=['ump_k%', 'ump_bb%', 'ump_r_per_game', 'ump_ba', 'ump_obp', 'ump_slg', 
                                       'ump_k_boost', 'ump_bb_boost', 'ump_r_boost', 'ump_ba_boost', 'ump_obp_boost', 'ump_slg_boost'])

if len(umpire_data) > 1:
    avg_list = [sum(x) / len(umpire_data) for x in zip(*umpire_data)]
    
    umpire_data_df.loc[0] = avg_list

else:
    umpire_data_df.loc[0] = umpire_data[0]

print(umpire_data_df)

    

   ump_k%  ump_bb%  ump_r_per_game  ump_ba  ump_obp  ump_slg  ump_k_boost  \
0   0.223    0.081            8.61   0.246    0.313    0.402          1.0   

   ump_bb_boost  ump_r_boost  ump_ba_boost  ump_obp_boost  ump_slg_boost  
0          0.95         0.97          0.99           0.98           0.98  


Fetch Venue Stats:

*FUTURE CHANGES: DETERMINE A BETTER WAY TO SET DATA FOR SWITCH HITTERS.

In [48]:
#options = Options()
#options.add_argument("--headless")
#options.add_argument("--disable-gpu")
#options.add_argument("--no-sandbox")
#options.add_argument("--disable-dev-shm-usage")
#options.add_argument("--disable-extensions")
#options.add_argument("--disable-images")
#prefs = {
#    'profile.default_content_setting_values': {
#        'images': 2,
#        'javascript': 2,
#        'stylesheet': 2
#    }
#}
#options.add_experimental_option('prefs', prefs)
#
#service = Service(chrome_driver_path)
#driver = webdriver.Chrome(service=service, options=options)
#
#with webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) as driver:

try:
    driver.get('https://swishanalytics.com/mlb/mlb-park-factors')

    venue_element = WebDriverWait(driver, 12).until(EC.presence_of_element_located((By.XPATH, f"//h4[contains(text(), '{venue_name}')]")))

    venue_table = venue_element.find_element(By.XPATH, "./following::table[1]")

    venue_rows = venue_table.find_elements(By.TAG_NAME, 'tr')

    venue_data = []

    for row in venue_rows[1:]:
        venue_cells = row.find_elements(By.TAG_NAME, 'td')
        venue_row_data = [cell.text for cell in venue_cells]

        if batter_hand == 'L':
            venue_data.append(float(venue_row_data[0]))
        elif batter_hand == 'R':
            venue_data.append(float(venue_row_data[2]))
        else:
            venue_data.append(round((float(venue_row_data[0]) + float(venue_row_data[2])) / 2, 3))

except Exception as e:
    venue_data = [None for _ in range(9)]

#finally:
    #None
    #driver.close()
    #driver.quit()

print(venue_data)

[0.98, 0.71, 0.96, 0.93, 0.95, 0.99, 0.96, 0.98, 0.94]


In [49]:
venue_data_df = pd.DataFrame(columns=['ven_hr_x', 'ven_3B_x', 'ven_2B_x', 'ven_1B_x', 'ven_ba_x', 'ven_obp_x', 
                                      'ven_slg_x', 'ven_woba_x', 'ven_r_x'])

venue_data_df.loc[0] = venue_data

print(venue_data_df)

   ven_hr_x  ven_3B_x  ven_2B_x  ven_1B_x  ven_ba_x  ven_obp_x  ven_slg_x  \
0      0.98      0.71      0.96      0.93      0.95       0.99       0.96   

   ven_woba_x  ven_r_x  
0        0.98     0.94  


QUIT CHROME DRIVER AFTER SCRAPING COMPLETES

In [50]:
driver.close()
driver.quit()

Combine General Data into DataFrame

In [51]:
batter_general_data = {
    'player_first_name': first_name,
    'player_last_name': last_name,
    'player_id': player_id,
    'game_pk': game_pk,
    'game_status': game_status,
    'game_num': games_played + 1,
    'series_status': series_status,
    'date': og_date,
    'game_time': game_time,
    'temp': temp,
    'wind_speed': wind_speed,
    'venue_name': venue_name,
    'venue_id': venue_id,
    'team_name': team_name,
    'team_id': team_id,
    'opp_name': opp_name,
    'opp_id': opp_id,
    'home': home,
    'batter_hand': batter_hand,
    'batting_pos': batting_position,
    'batter_age': batter_age,
    'batter_year': batter_year,
    'batter_k_zone_top': batter_strike_zone_top,
    'batter_k_zone_bottom': batter_strike_zone_bottom,
    's_pitcher_first_name': sp_first_name,
    's_pitcher_last_name': sp_last_name,
    's_pitcher_id': sPitcher_id,
    's_pitcher_hand': sPitcher_hand,
}

if len(proj_ump_ids) < 1:
    batter_general_data['ump_id'] = None
    batter_general_data['ump_name'] = None

else:
    batter_general_data['ump_id'] = proj_ump_ids[0]
    batter_general_data['ump_name'] = proj_umps[0]

batter_general_data_df = pd.DataFrame([batter_general_data])

print(batter_general_data_df)

  player_first_name player_last_name  player_id  game_pk  game_status  \
0            shohei           ohtani     660271   745386         True   

   game_num  series_status        date  game_time  temp  wind_speed  \
0       106           True  2024-07-31       2040    73           8   

   venue_name  venue_id            team_name  team_id          opp_name  \
0  Petco Park      2680  Los Angeles Dodgers      119  San Diego Padres   

   opp_id   home batter_hand  batting_pos  batter_age  batter_year  \
0     135  False           L            1          30            6   

   batter_k_zone_top  batter_k_zone_bottom s_pitcher_first_name  \
0               3.51                  1.61                dylan   

  s_pitcher_last_name  s_pitcher_id s_pitcher_hand  ump_id ump_name  
0               cease        656302              R  521251  Ben May  


Merge Batter General Data and Batter Season Data

In [52]:
batter_stats_df = pd.concat([batter_general_data_df, batter_game_stats_df, batter_season_stats_df, series_stats_df], axis=1)

print(batter_stats_df)

  player_first_name player_last_name  player_id  game_pk  game_status  \
0            shohei           ohtani     660271   745386         True   

   game_num  series_status        date  game_time  temp  wind_speed  \
0       106           True  2024-07-31       2040    73           8   

   venue_name  venue_id            team_name  team_id          opp_name  \
0  Petco Park      2680  Los Angeles Dodgers      119  San Diego Padres   

   opp_id   home batter_hand  batting_pos  batter_age  batter_year  \
0     135  False           L            1          30            6   

   batter_k_zone_top  batter_k_zone_bottom s_pitcher_first_name  \
0               3.51                  1.61                dylan   

  s_pitcher_last_name  s_pitcher_id s_pitcher_hand  ump_id ump_name  pa  ab  \
0               cease        656302              R  521251  Ben May   4   3   

   h  r  rbi  hr  tb  k  bb  pa_last_7  ab_last_7  h_last_7  r_last_7  \
0  0  0    0   0   0  0   1   4.714286   3.857143  

Merge bv Stats

In [53]:
bv_stats_df = pd.concat([bvp_data_df, bv_opp_sp_df, bv_opp_df], axis=1)

print(bv_stats_df)

  bvp_games_faced bvp_ip_total bvp_ip_per_game bvp_p_total bvp_p_per_game  \
0            None         None            None        None           None   

  bvp_pa_total bvp_pa_per_game bvp_h_total bvp_h_per_game bvp_hr_total  \
0         None            None        None           None         None   

  bvp_hr_per_game bvp_tb_total bvp_tb_per_game bvp_k_total bvp_k_per_game  \
0            None         None            None        None           None   

  bvp_k% bvp_woba bvp_ba bv_opp_sp_pa bv_opp_sp_ab bv_opp_sp_h bv_opp_sp_r  \
0   None     None   None         None         None        None        None   

  bv_opp_sp_rbi bv_opp_sp_hr bv_opp_sp_tb bv_opp_sp_bb bv_opp_sp_k  \
0          None         None         None         None        None   

  bv_opp_sp_ba bv_opp_sp_obp bv_opp_sp_slg  bv_opp_games_played  bv_opp_pa  \
0         None          None          None                    8       4.75   

   bv_opp_ab  bv_opp_h  bv_opp_r  bv_opp_rbi  bv_opp_hr  bv_opp_tb  bv_opp_bb  \
0    

Merge Pitching Stats

In [54]:
pitching_stats_df = pd.concat([sp_season_stats_df, sp_vs_team_stats_df], axis=1)

print(pitching_stats_df)

   sp_games_started  sp_ip_per_game  sp_p_per_game  sp_s_per_game     sp_s%  \
0                22        5.954545      99.818182      63.227273  0.633424   

   sp_bf_per_game  sp_ab_per_game  sp_h_per_game  sp_r_per_game  \
0            23.5       21.181818       4.045455       2.454545   

   sp_er_per_game  sp_hr_per_game  sp_tb_per_game  sp_bb_per_game  \
0        2.318182        0.681818        7.227273        1.818182   

   sp_k_per_game     sp_k%  sp_to_per_game  sp_era  sp_whip_per_game  \
0       7.636364  0.324952       17.863636     3.5          1.182781   

   sp_whip_szn    sp_baa sp_vs_team_ip sp_vs_team_p sp_vs_team_s  \
0     0.984733  0.190987          None         None         None   

  sp_vs_team_s% sp_vs_team_bf sp_vs_team_ab sp_vs_team_h sp_vs_team_r  \
0          None          None          None         None         None   

  sp_vs_team_er sp_vs_team_hr sp_vs_team_tb sp_vs_team_bb sp_vs_team_k  \
0          None          None          None          None       

Merge Team Stats

In [55]:
team_stats_df = pd.concat([opp_stats_df, team_vs_opp_df], axis=1)

print(team_stats_df)

   opp_num_pitchers_per_game  opp_p_per_game  opp_s_per_game    opp_s%  \
0                   4.266055      146.385321       94.311927  0.644272   

   opp_bf_per_game  opp_ab_per_game  opp_h_per_game  opp_r_per_game  \
0        37.100917        33.412844        8.100917        4.293578   

   opp_er_per_game  opp_hr_per_game  opp_tb_per_game  opp_bb_per_game  \
0              4.0         1.082569        13.238532          2.87156   

   opp_k_per_game    opp_k%  opp_to_per_game  opp_era  opp_whip   opp_baa  \
0        8.798165  0.237141        26.537615     4.07  1.240407  0.242449   

   team_vs_opp_games_played  team_vs_opp_num_batters  team_vs_opp_pa  \
0                         9                 9.666667       40.222222   

   team_vs_opp_ab  team_vs_opp_h  team_vs_opp_r  team_vs_opp_rbi  \
0       34.444444       7.111111       4.666667         4.555556   

   team_vs_opp_hr  team_vs_opp_tb  team_vs_opp_bb  team_vs_opp_k  \
0             1.0       11.222222        4.666667       

Merge Umpire and Venue Stats

In [56]:
factor_stats_df = pd.concat([umpire_data_df, venue_data_df], axis=1)

print(factor_stats_df)

   ump_k%  ump_bb%  ump_r_per_game  ump_ba  ump_obp  ump_slg  ump_k_boost  \
0   0.223    0.081            8.61   0.246    0.313    0.402          1.0   

   ump_bb_boost  ump_r_boost  ump_ba_boost  ump_obp_boost  ump_slg_boost  \
0          0.95         0.97          0.99           0.98           0.98   

   ven_hr_x  ven_3B_x  ven_2B_x  ven_1B_x  ven_ba_x  ven_obp_x  ven_slg_x  \
0      0.98      0.71      0.96      0.93      0.95       0.99       0.96   

   ven_woba_x  ven_r_x  
0        0.98     0.94  


FORM COMPLETE RAW DATA DF

In [57]:
complete_data_df = pd.concat([batter_stats_df, bv_stats_df, pitching_stats_df, team_stats_df, factor_stats_df], axis=1)

print(complete_data_df) 

  player_first_name player_last_name  player_id  game_pk  game_status  \
0            shohei           ohtani     660271   745386         True   

   game_num  series_status        date  game_time  temp  wind_speed  \
0       106           True  2024-07-31       2040    73           8   

   venue_name  venue_id            team_name  team_id          opp_name  \
0  Petco Park      2680  Los Angeles Dodgers      119  San Diego Padres   

   opp_id   home batter_hand  batting_pos  batter_age  batter_year  \
0     135  False           L            1          30            6   

   batter_k_zone_top  batter_k_zone_bottom s_pitcher_first_name  \
0               3.51                  1.61                dylan   

  s_pitcher_last_name  s_pitcher_id s_pitcher_hand  ump_id ump_name  pa  ab  \
0               cease        656302              R  521251  Ben May   4   3   

   h  r  rbi  hr  tb  k  bb  pa_last_7  ab_last_7  h_last_7  r_last_7  \
0  0  0    0   0   0  0   1   4.714286   3.857143  

Create a TempCSV for Testing

In [58]:
#if not complete_data_df.empty:
#
#    with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix='.csv') as tmp_file:
#        complete_data_df.to_csv(tmp_file.name, index=False)
#        print(f'Temporary CSV file creat3ed at {tmp_file.name}')
#        
#else:
#    print('No data was found.')

Temporary CSV file creat3ed at C:\Users\cbhal\AppData\Local\Temp\tmp2pb9ajty.csv
